In [36]:
!pip install tensorflow numpy pandas nltk openpyxl

In [37]:
import pandas as pd
import numpy as np
import tensorflow as tf
import re
import string
import nltk
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Embedding, Dense

# Download necessary NLP resources
nltk.download('punkt')


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\nkrid\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [38]:
# Load dataset (Ensure the file is in the same directory as your notebook)
file_path = "mental_health_chatbot_dataset.xlsx"
df = pd.read_excel(file_path)

# Show dataset preview
df.head()

,ID,User Input,Bot Response,Intent,Sentiment,Context
0,1,I feel really down today.,I'm sorry to hear that. Do you want to talk ab...,Sadness,Negative,Depression
1,2,I had a panic attack.,That sounds tough. Try to take deep breaths. I...,Anxiety,Negative,Panic Attack
2,3,I'm so stressed with work.,That sounds overwhelming. Would you like to tr...,Stress,Negative,Work Stress
3,4,I can't sleep at night.,I'm here to help. Have you tried deep breathin...,Insomnia,Negative,Sleep Issues
4,5,I'm feeling better today!,That's great! What helped you feel better?,Happiness,Positive,General


In [39]:
# Function to clean text
def preprocess_text(text):
    text = str(text).lower()  # Convert to lowercase
    text = re.sub(f"[{string.punctuation}]", "", text)  # Remove punctuation
    text = nltk.word_tokenize(text)  # Tokenize
    return " ".join(text)

# Apply preprocessing to user inputs and bot responses
df["User Input"] = df["User Input"].apply(preprocess_text)
df["Bot Response"] = df["Bot Response"].apply(preprocess_text)

# Display cleaned data
df.head()

,ID,User Input,Bot Response,Intent,Sentiment,Context
0,1,i feel really down today,im sorry to hear that do you want to talk abou...,Sadness,Negative,Depression
1,2,i had a panic attack,that sounds tough try to take deep breaths im ...,Anxiety,Negative,Panic Attack
2,3,im so stressed with work,that sounds overwhelming would you like to try...,Stress,Negative,Work Stress
3,4,i cant sleep at night,im here to help have you tried deep breathing ...,Insomnia,Negative,Sleep Issues
4,5,im feeling better today,thats great what helped you feel better,Happiness,Positive,General


In [ ]:
# Define hyperparameters
MAX_LEN = 20  # Maximum sequence length
VOCAB_SIZE = 5000  # Limit vocabulary size

# Create a tokenizer
tokenizer = Tokenizer(num_words=VOCAB_SIZE, filters="")
tokenizer.fit_on_texts(df["User Input"].tolist() + df["Bot Response"].tolist())

# Convert text to sequences
input_sequences = tokenizer.texts_to_sequences(df["User Input"])
response_sequences = tokenizer.texts_to_sequences(df["Bot Response"])

# Apply padding
input_sequences = pad_sequences(input_sequences, maxlen=MAX_LEN, padding="post")
response_sequences = pad_sequences(response_sequences, maxlen=MAX_LEN, padding="post")

# Convert to NumPy arrays
input_sequences = np.array(input_sequences)
response_sequences = np.array(response_sequences)

# Check shapes
input_sequences.shape, response_sequences.shape


In [41]:
# Define encoder
encoder_inputs = Input(shape=(max_len,))
encoder_embedding = Embedding(VOCAB_SIZE, 128)(encoder_inputs)
encoder_lstm = LSTM(256, return_state=True)
encoder_outputs, state_h, state_c = encoder_lstm(encoder_embedding)

# Define decoder
decoder_inputs = Input(shape=(MAX_LEN,))
decoder_embedding = Embedding(VOCAB_SIZE, 128)(decoder_inputs)
decoder_lstm = LSTM(256, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_embedding, initial_state=[state_h, state_c])

# Dense output layer
decoder_dense = Dense(VOCAB_SIZE, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

# Build and compile model
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
model.compile(optimizer="adam", loss="sparse_categorical_crossentropy")

# Show model summary
model.summary()


Model: "functional_7"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_6       │ (None, 20)        │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_layer_7       │ (None, 20)        │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding_6         │ (None, 20, 128)   │    640,000 │ input_layer_6[0]… │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding_7         │ (None, 20, 128)   │    640,000 │ input_layer_7[0]… │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm_6 (LSTM)       │ [(None, 256),     │    394,240 │ embedding_6[0][0] │
│                     │ (None, 256),      │            │                   │
│                     │ (None, 256)]      │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm_7 (LSTM)       │ [(None, 20, 256), │    394,240 │ embedding_7[0][0… │
│                     │ (None, 256),      │            │ lstm_6[0][1],     │
│                     │ (None, 256)]      │            │ lstm_6[0][2]      │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_3 (Dense)     │ (None, 20, 5000)  │  1,285,000 │ lstm_7[0][0]      │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 3,353,480 (12.79 MB)

 Trainable params: 3,353,480 (12.79 MB)

 Non-trainable params: 0 (0.00 B)

In [42]:
# Train the model
model.fit([input_sequences, response_sequences], response_sequences, batch_size=64, epochs=50, validation_split=0.2)

Epoch 1/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 4s 4s/step - loss: 8.5180 - val_loss: 8.5037
Epoch 2/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 195ms/step - loss: 8.5021 - val_loss: 8.4867
Epoch 3/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 189ms/step - loss: 8.4829 - val_loss: 8.4607
Epoch 4/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 193ms/step - loss: 8.4530 - val_loss: 8.4135
Epoch 5/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 176ms/step - loss: 8.3982 - val_loss: 8.3127
Epoch 6/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 192ms/step - loss: 8.2812 - val_loss: 8.0725
Epoch 7/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 170ms/step - loss: 8.0101 - val_loss: 7.6323
Epoch 8/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 193ms/step - loss: 7.5239 - val_loss: 7.1788
Epoch 9/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 236ms/step - loss: 7.0021 - val_loss: 6.7962
Epoch 10/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 208ms/step - loss: 6.5426 - val_loss: 6.4351
Epoch 11/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 246ms/step - loss: 6.1045 - val_loss: 6.0734
Epoch 12/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 196ms/step - loss: 5.6633 - val_loss:

In [43]:
import h5py
print("h5py is installed. Version:", h5py.__version__)


h5py is installed. Version: 3.13.0


In [ ]:
model.save("mental_health_chatbot.keras")
print("Model Successfully Saved!!")

In [45]:
from tensorflow.keras.models import load_model

# Load the trained chatbot model
model = load_model("mental_health_chatbot.keras")

print("Model loaded successfully!")


Model loaded successfully!


In [46]:
import tensorflow as tf
print("TensorFlow Version:", tf.__version__)

TensorFlow Version: 2.16.1


In [47]:
from tensorflow.keras.models import load_model

# Load the trained chatbot model
model = load_model("mental_health_chatbot.keras")  # Use .keras format for Keras 3.x

In [48]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
import pickle


In [ ]:
from tensorflow.keras.models import load_model
import numpy as np

# Load tokenizer
with open("tokenizer.pkl", "rb") as f:
    tokenizer = pickle.load(f)

# Load trained model
model = load_model("mental_health_chatbot.keras")

# Define function to generate chatbot responses
def get_bot_response(user_input):
    user_seq = tokenizer.texts_to_sequences([user_input])
    user_seq = pad_sequences(user_seq, maxlen=max_len, padding="post")  # Ensure consistent length

    # Pass user input through the model
    prediction = model.predict([user_seq, np.zeros((1, max_len))])  # Ensure decoder input is correct
    predicted_index = np.argmax(prediction, axis=-1)

    # Convert predicted sequence back to words
    response_words = [word for word, index in tokenizer.word_index.items() if index in predicted_index[0]]

    return " ".join(response_words)


# Run chatbot loop
while True:
    print(max_len)
    user_input = input("You: ")
    if user_input.lower() in ["exit", "quit"]:
        print("Chatbot: Goodbye!")
        break
    response = get_bot_response(user_input)
    print("Chatbot:", response)
